## ControlFlow Example 1

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import controlflow as cf



The parameters for `run()` function are:

- objective
- turn_strategy
- max_llm_calls
- max_agent_turns
- raise_on_failure
- handlers
- model_kwargs
- run_until

In [6]:
# Sample emails to process
emails = [
    "Hello, I need an update on the project status.",
    "Subject: Exclusive offer just for you!",
    "Urgent: Project deadline moved up by one week.",
]

# Define email parameters
email_params = {
    "sender_name": "John Smith",
    "your_name": "Alice Johnson",
    "your_title": "Project Manager",
    "company_name": "Tech Corp"
}

# Define a simple task to generate a reply
def generate_reply(email, params):
    return cf.run(
        objective="Generate a reply to the email",  # Objective of the task
        context={
            "email": email,
            "sender_name": params["sender_name"],
            "your_name": params["your_name"],
            "your_title": params["your_title"],
            "company_name": params["company_name"]
        }    )

# Run the task for each email
for email in emails:
    reply = generate_reply(email, email_params)
    print(f"Reply: {reply}")

Output()

Reply: Subject: Project Status Update

Hi John,

I hope this message finds you well. I wanted to provide you with an update on the project status as you requested. We are currently on track and making good progress. The team is working diligently to meet our deadlines and ensure everything is aligned with the project goals.

If you have any specific questions or need further details, please feel free to reach out.

Best regards,

Alice Johnson  
Project Manager  
Tech Corp


Output()

Reply: Subject: Re: Exclusive offer just for you!

Hi John,

Thank you for reaching out with this exclusive offer. I am eager to learn more about it and see how it might benefit our projects at Tech Corp. Could you please provide more details about the offer? I want to ensure we make the most informed decision.

Looking forward to your response.

Best regards,

Alice Johnson  
Project Manager  
Tech Corp


Output()

Reply: Subject: Re: Urgent: Project deadline moved up by one week.

Hi John,

Thank you for informing me about the change in the project deadline. I understand the urgency and will immediately coordinate with the team to realign our schedule accordingly. Please let me know if there are any specific areas you would like us to prioritize or if there is additional support needed from your side.

Best regards,

Alice Johnson
Project Manager
Tech Corp


---

### Create an agent for the task

This code demonstrates an automated email processing system using ControlFlow agents.

It creates two specialized AI agents:
1. Email Classifier - Uses GPT-4 to categorize emails as either 'important' or 'spam'
2. Email Responder - Uses GPT-4 to generate professional responses for important emails

The main process_email() flow function:
- Takes an email string as input
- Uses the classifier agent to categorize the email
- For important emails, uses the responder agent to generate a response
- For spam emails, returns a standard "no response needed" message


In [8]:
classifier = cf.Agent(
    name="Email Classifier",
    model="openai/gpt-4",  # You can adjust the model based on your needs
    instructions="You are an expert at quickly classifying emails. Always "
                "respond with exactly one word: either 'important' or 'spam'."
)

responder = cf.Agent(
    name="Email Responder",
    model="openai/gpt-4",
    instructions="You are an expert at crafting professional email responses. "
                "Your replies should be concise but friendly."
)

# Create the email processing flow
@cf.flow
def process_email(email_content: str):
    # Step 1: Classify the email
    category = cf.run(
        objective="Classify this email",
        result_type=["important", "spam"],
        agents=[classifier],
        context={"email": email_content}
    )
    
    print(f"Email classified as: {category}")

    # Step 2: Generate response for important emails
    if category == "important":
        response = cf.run(
            objective="Write a response to this important email",
            result_type=str,
            agents=[responder],
            context={"email": email_content,
                    "sender_name": email_params["sender_name"],
                    "your_name": email_params["your_name"],
                    "your_title": email_params["your_title"],
                    "company_name": email_params["company_name"] 
            }
        )
        return response
    else:
        return "No response needed for spam email."

# Process all emails
print("Processing emails...\n")
for i, email in enumerate(emails, 1):
    print(f"\nEmail {i}: {email}")
    response = process_email(email)
    print(f"Response: {response}\n")
    print("-" * 50)

Processing emails...


Email 1: Hello, I need an update on the project status.


Output()

Email classified as: important


Output()

Response: Dear John,

Thank you for reaching out for an update on the project. I'm pleased to let you know that we are making good progress. We have completed most of the tasks and are on schedule. I'll provide a detailed update by the end of the day.

Please let me know if there's anything else you need in the meantime.

Best regards,
Alice Johnson
Project Manager
Tech Corp

--------------------------------------------------

Email 2: Subject: Exclusive offer just for you!


...

Email classified as: spam
Response: No response needed for spam email.

--------------------------------------------------

Email 3: Urgent: Project deadline moved up by one week.


Output()

Email classified as: important


Output()

Response: Dear John,

Thank you for your email and for bringing this urgent matter to my attention.

I understand that the project deadline has been moved up by a week. This is indeed a significant change, and we will do our best to adjust our work schedule to meet this new deadline.

Your continued support and understanding are greatly appreciated as we navigate this change. If there are any further details or specific requirements related to this new deadline, please do not hesitate to share.

Best regards,
Alice Johnson
Project Manager
Tech Corp

--------------------------------------------------
